In [47]:
import requests
from bs4 import BeautifulSoup
import tqdm
import sys
import pandas as pd
sys.path.append('../')
from utility.file_utility import FileUtility
from pandas import Series
from bdpAPI.bdpcall import BDPCall

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

global meta_data_paths
meta_data_paths={'allbiblexls':'0meta/main.xls','iso_file':'meta/ISO639-3.tsv', 'final_file': '0meta/meta.tsv'}


def getMetaEbible():
    base_url = 'http://ebible.org/Scriptures/copyright.php'
    soup = BeautifulSoup(requests.get(base_url).content)
    tables=[]
    for link in soup.select('table'):
        tables.append(pd.read_html(str(link),header=0)[0])
    results=pd.concat(tables,sort=True)


    mask=(results['FCBH/DBS'].str.len() == 6) & (results['FCBH/DBS'].str.isupper())
    df = results.loc[mask]
    df['iso']=[x[0:3] for x in df['ID'].tolist()]
    df=df[['iso','FCBH/DBS','Language','Language in English', 'Year','Short Title']]
    return df

def getMetaFindBible():
    global meta_data_paths

    df_main=pd.read_excel(meta_data_paths['allbiblexls'])
    df_main=df_main.rename(index=str, columns={"code": "FCBH/DBS", "language": "Language",'description':'Short Title','date':'Year'})
    return df_main


def getMetaMerged():
    global meta_data_paths
    
    # read and merge two bibles
    df_ebible=getMetaEbible()
    df_main=getMetaFindBible()
    df=pd.concat([df_ebible[['iso','FCBH/DBS','Language','Year','Short Title']],df_main[['iso','FCBH/DBS','Language','Year','Short Title']]])
    df=df.rename(index=str, columns={"Short Title": "description",'iso':'ISO'})
    
    # get metadata from ISO Rec
    df_iso=pd.read_table(meta_data_paths['iso_file'])
    iso_dict=Series(df.Language.values,index=df.ISO).to_dict()
    iso_dict.update(Series(df_iso.NAM_LABEL.values,index=df_iso.ISO).to_dict())
    df["Language"] = df["ISO"].map(lambda x:iso_dict[x])
    
    # set the key
    df=df.set_index('FCBH/DBS')
    return df

def create_meta_data(apikey='f03a423aad95120f8eb40005070f19e9',path='/mounts/data/proj/asgari/final_proj/000_datasets/testbib/API_new_Res'):
    global meta_data_paths
    df=getMetaMerged()
    # check on the API
    BDP_API=BDPCall(apikey)
    IDS=df.index.tolist()
    IDS.sort()
    existOnAPI=BDP_API.doesExist_bibleIDS(path,IDS,20)
    df['exist_API']=df.index.map(lambda x:existOnAPI[x])
    df.to_csv(meta_data_paths['final_file'] ,sep='\t')

    
def update_meta_data(apikey='f03a423aad95120f8eb40005070f19e9',path='/mounts/data/proj/asgari/final_proj/000_datasets/testbib/API_new_Res'):
    global meta_data_paths
    df=pd.read_table(meta_data_paths['final_file'])
    # check on the API
    BDP_API=BDPCall(apikey)
    IDS=df.index.tolist()
    IDS.sort()
    existOnAPI=BDP_API.doesExist_bibleIDS(path,IDS,20)
    df['exist_API']=df.index.map(lambda x:existOnAPI[x])
    df.to_csv(meta_data_paths['final_file'] ,sep='\t')

    
def crawl_bibles(apikey='f03a423aad95120f8eb40005070f19e9',path='/mounts/data/proj/asgari/final_proj/000_datasets/testbib/API_new_Res'):
    BDP_obj = BDPCall(apikey)
    BDP_obj.crawl_bible_books(path,10)
    

In [ ]:
https://dbt.io/country/countrylang?key=f03a423aad95120f8eb40005070f19e9&v=2


In [48]:
import requests
from bs4 import BeautifulSoup
import tqdm
import sys
sys.path.append('../')
from utility.file_utility import FileUtility



base_url = 'https://bible.cloud/inscript/content/texts/'
soup = BeautifulSoup(requests.get(base_url).content)

# collect the game ids
collect_version_links=[]
langs=dict()
url_dict=dict()
mapping=dict()
for link in tqdm.tqdm(soup.select('a')):
    if 'class' in link.attrs:
        if len(link.contents)==1:
            if 'href' in link.attrs:
                url_dict[link.contents[0].replace(' ','-')]='https://bible.cloud/inscript/content/texts/'+link.attrs['href'].replace('index.html','MT1.html')
                mapping[link.attrs['href'].split('/')[0]]='https://bible.cloud/inscript/content/texts/'+link.attrs['href'].replace('index.html','MT1.html')
                langs[link.attrs['href'].split('/')[0]]=(link.contents[0].replace(' ','-'))

100%|██████████| 1581/1581 [00:00<00:00, 397577.47it/s]


In [59]:
exist=[]
for x in tqdm.tqdm(mapping):
    response = requests.get('https://bible.cloud/inscript/content/texts/'+x+'/index.html')
    if response.status_code==200:
        exist.append(x)



100%|██████████| 790/790 [07:21<00:00,  1.79it/s]


In [62]:
df=pd.read_table('/mounts/data/proj/asgari/final_proj/000_datasets/testbib/API/reports/crawl_report_API.tsv')

api=df[df['verses']>0]['trans_ID'].tolist()

In [68]:
cloud_complement=[x for x in exist if x not in api]

In [71]:
df_main=pd.read_table('meta/meta.tsv')
columns={"FCBH/DBS":[], "Language":[],'Description':[],'Year':[]}
df_iso=pd.read_table(meta_data_paths['iso_file'])
iso_dict=Series(df_iso.NAM_LABEL.values,index=df_iso.ISO).to_dict()

new=[]
for x in tqdm.tqdm(cloud_complement):
    if not (df_main['FCBH/DBS']==x).tolist().count(True)>0:
        response = requests.get('https://bible.cloud/inscript/content/texts/'+x+'/index.html')
        if response.status_code == 200:
            try:
                columns["Language"].append(iso_dict[x[0:3].lower()])
                columns["FCBH/DBS"].append(x)
                columns["Description"].append(langs[x])        
                columns["Year"].append('NAN')        
            except:
                print(x)

 93%|█████████▎| 382/412 [00:13<00:01, 27.65it/s] 

SPNNBL


 93%|█████████▎| 383/412 [00:14<00:01, 26.68it/s]

SPNR60


 93%|█████████▎| 384/412 [00:14<00:01, 25.77it/s]

SPNR09


100%|██████████| 412/412 [00:15<00:00, 26.65it/s]


In [72]:
pd.DataFrame(columns)

,FCBH/DBS,Language,Description,Year
0,GRCTIS,Ancient Greek,Η-Καινή-Διαθήκη,NAN
1,WARAB4,Waray-Waray,An-Baraan-na-Kasuratan,NAN
2,BCLMBB,Central Bicolano,Marahay-na-Bareta-Biblia,NAN
3,BCLMBC,Central Bicolano,Marahay-na-Bareta-Biblia-+-Deuterocanon,NAN
4,CEBAB1,Cebuano,Ang-Biblia,NAN
5,CEBMBP,Cebuano,Maayong-Balita-Biblia,NAN
6,CEBMBC,Cebuano,Maayong-Balita-Biblia-Adunay-Deuterocanon,NAN
7,HILAB2,Hiligaynon,Ang-Biblia,NAN
8,HILMBB,Hiligaynon,Maayong-Balita-nga-Biblia,NAN
9,HILMBC,Hiligaynon,Maayong-Balita-nga-Biblia-+-Deuterocanon,NAN


In [41]:
    global meta_data_paths
    
    # read and merge two bibles
    df_ebible=getMetaEbible()
    df_main=getMetaFindBible()
    df=pd.concat([df_ebible[['iso','FCBH/DBS','Language','Year','Short Title']],df_main[['iso','FCBH/DBS','Language','Year','Short Title']]])
    df=df.rename(index=str, columns={"Short Title": "description",'iso':'ISO'})
    
    # get metadata from ISO Rec
    df_iso=pd.read_table(meta_data_paths['iso_file'])
    iso_dict=Series(df.Language.values,index=df.ISO).to_dict()
    iso_dict.update(Series(df_iso.NAM_LABEL.values,index=df_iso.ISO).to_dict())
    df["Language"] = df["ISO"].map(lambda x:iso_dict[x])
    
    # set the key

[('BEABFB',
  ('OOTECH-OOCHU-TAKEHNIYA-TINKLES-ST.-MARK',
   <a class="latin" href="BEABFB/index.html">OOTECH OOCHU TAKEHNIYA TINKLES ST. MARK</a>)),
 ('CHNCU1',
  ('新标点和合本',
   <a class="bitstream_cyberbit" href="CHNCU1/index.html">新标点和合本</a>)),
 ('CHNCUV',
  ('新標點和合本',
   <a class="bitstream_cyberbit" href="CHNCUV/index.html">新標點和合本</a>)),
 ('COPBHC',
  ('القبطية-البحيرية-العهد-الجديد',
   <a class="new_athena_unicode" href="COPBHC/index.html">القبطية البحيرية العهد الجديد</a>)),
 ('COPCNT',
  ('القبطية-العهد-الجديد',
   <a class="new_athena_unicode" href="COPCNT/index.html">القبطية العهد الجديد</a>)),
 ('BREBRG',
  ('an-Bibl', <a class="latin" href="BREBRG/index.html">an Bibl</a>)),
 ('FRELSG',
  ('Louis-Segond-1910',
   <a class="latin" href="FRELSG/index.html">Louis Segond 1910</a>)),
 ('GERL12',
  ('Luther-Bibel-1912',
   <a class="latin" href="GERL12/index.html">Luther Bibel 1912</a>)),
 ('GRCTIS',
  ('Η-Καινή-Διαθήκη',
   <a class="gentium_plus" href="GRCTIS/index.html">Η Καινή

In [17]:
df2=pd.read_table('/mounts/data/proj/asgari/final_proj/000_datasets/testbib/API/reports/crawl_report_API.tsv')



In [18]:
df2

,Unnamed: 0,FCBH/DBS,ISO,Language,Year,description,exist_API,verses
0,1,AAUWBT,aau,Abau,2006,Abau NT,True,NaN
1,2,ACRWBT,acr,Rabinal Achí,2009,Achi Rabinal NT (new orthography),True,NaN
2,4,ACCIBS,acr,Rabinal Achí,2009,New Testament in Achi,True,0.0
3,5,ACUTBL,acu,Achuar-Shiwiar,2008,Achuar-shiwiar Bible,True,NaN
4,10,AGUNVS,agu,Aguacateco,2011,Aguacateco Bible,True,0.0
5,12,AGNWPS,agn,Agutaynen,2004,Agutaynen NT,True,NaN
6,15,AKEBSS,ake,Akawaio,2010,Akawaio NT,True,NaN
7,16,BSSWBT,bss,Akoose,2011,Akoose NT,True,NaN
8,19,GAHPNG,gah,Alekano,"1973, 1986",Alekano Bible,True,NaN
9,21,ALPWBT,alp,Alune,2012,Alune NT,True,NaN


In [133]:
code='NANTTV'
df1=pd.read_table('meta/meta.tsv')
iso_dict = Series(df1['ISO'].values, index=df1['FCBH/DBS']).to_dict()

In [135]:
str(iso_dict[code])

'nan'

In [116]:
df1=pd.read_table('meta/meta.tsv')
df2=pd.read_table('/mounts/data/proj/asgari/final_proj/000_datasets/testbib/API/reports/crawl_report_API.tsv')


In [129]:
IDS1=df1[df1['exist_API']==True][df1['exist_BibleIS']==True]['FCBH/DBS'].tolist()
IDS2=df2[df2['verses']==0]['FCBH/DBS'].tolist()

In [132]:
set(IDS1).intersection(IDS2)

{'AAKWBT',
 'AAUWBT',
 'ABIWBT',
 'ABYWBT',
 'ACCIBS',
 'ADHBSU',
 'AFBGNV',
 'AGRTBL',
 'AGUNVS',
 'AIWDPI',
 'AKPWBT',
 'ALDABC',
 'ALWDPI',
 'AMNPNG',
 'ANYWBT',
 'APEWBT',
 'APRWBT',
 'ARBIBS',
 'ARBWTC',
 'ARYMBS',
 'ASESLV',
 'ASOTBL',
 'AVAIBT',
 'AWAIBS',
 'AWNDPI',
 'AYMBSB',
 'AZBEMV',
 'AZGTBL',
 'BAMLSB',
 'BAVWBT',
 'BBABSB',
 'BBJABC',
 'BBOBSM',
 'BCIUBS',
 'BDHWBT',
 'BDQDVS',
 'BEFWBT',
 'BEHWBT',
 'BEMUBS',
 'BEPLAI',
 'BEVABU',
 'BFABSS',
 'BHLPNG',
 'BJVWYI',
 'BKDWBT',
 'BKLLAI',
 'BKMBSC',
 'BKYBSN',
 'BLHLBT',
 'BMVCAB',
 'BNOWBT',
 'BOJWBT',
 'BONPNG',
 'BOXWYI',
 'BQCSIM',
 'BSCWBT',
 'BSNTBL',
 'BSQLBS',
 'BULB40',
 'BULBBS',
 'BVCWBT',
 'BWQBFO',
 'BWRBSN',
 'BXGDRC',
 'BXRIBT',
 'BYVABC',
 'BZABSL',
 'CAGSBP',
 'CASNTM',
 'CAXSBB',
 'CBTTBL',
 'CCOTBL',
 'CCPBIV',
 'CEBPVV',
 'CHFTBL',
 'CHJBSM',
 'CHVIBT',
 'CKKWBT',
 'CLYAVA',
 'CNHBSM',
 'CNLTBL',
 'CNMGVR',
 'CNMRGB',
 'CNWBSM',
 'COTTBL',
 'COUWBT',
 'CPATBL',
 'CPUWBT',
 'CTGBSB',
 'CTPTBL',
 'CZTBIV',

In [123]:
codes=set(df2[df2['verses']==0]['FCBH/DBS'].tolist()+df1[df1['exist_API']==False]['FCBH/DBS'].tolist())
codes=set(df2[df2['verses']==0]['FCBH/DBS'].tolist()+df1[df1['exist_API']==False]['FCBH/DBS'].tolist())

In [114]:
import requests

response = requests.get('http://www.bible.is/AAKWBT/Matt/1')

In [115]:
response.status_code

404

In [28]:
len(set(df[df['exist_API']]['ISO'].tolist()))

1088

In [49]:
book_map=dict()
for l in FileUtility.load_list('/mounts/data/proj/asgari/final_proj/1000langs/data_config/books2numbers.txt'):
    for y in l.split('\t')[1].split(','):
        book_map[y]=l.split('\t')[0] 

In [54]:
import codecs

f=

In [76]:
import json

books = []
for line in codecs.open('/mounts/data/proj/asgari/final_proj/000_datasets/testbib/API_new_Res/ENGWEB/content.txt','r','utf-8'):
    books.append(json.loads(line))
    
#a=json.loads(codecs.open('/mounts/data/proj/asgari/final_proj/000_datasets/testbib/API_new_Res/ENGWEB/content.txt','r','utf-8').read())
#rec=a[0]
#book_mapping[rec['book_id']]+

In [77]:
bible=dict()
for book in books:
    for rec in book:
        bible[book_map[rec['book_id']]+rec['chapter_id'].zfill(3)+rec['verse_id'].zfill(3)]=rec['verse_text'].strip()
        

In [90]:
import collections

od = collections.OrderedDict(sorted(bible.items()))

In [89]:
for x,y in bible.items():
    if x not in Michaels:
        print(y)

In the beginning, God created the heavens and the earth.
The earth was formless and empty. Darkness was on the surface of the deep and God’s Spirit was hovering over the surface of the waters.
God said, “Let there be light, ” and there was light.
God saw the light, and saw that it was good. God divided the light from the darkness.
God called the light “day”, and the darkness he called “night”. There was evening and there was morning, the first day.
God said, “Let there be an expanse in the middle of the waters, and let it divide the waters from the waters.”
God made the expanse, and divided the waters which were under the expanse from the waters which were above the expanse; and it was so.
God called the expanse “sky”. There was evening and there was morning, a second day.
God said, “Let the waters under the sky be gathered together to one place, and let the dry land appear”; and it was so.
God called the dry land “earth”, and the gathering together of the waters he called “seas”. God 

You shall have just balances, just weights, a just ephah, and a just hin. I am Yahweh your God, who brought you out of the land of Egypt.
“‘You shall observe all my statutes, and all my ordinances, and do them. I am Yahweh.’”
Yahweh spoke to Moses, saying,
“Moreover, you shall tell the children of Israel, ‘Anyone of the children of Israel, or of the strangers who live as foreigners in Israel, who gives any of his offspring to Molech; he shall surely be put to death. The people of the land shall stone him with stones.
I also will set my face against that person, and will cut him off from among his people because he has given of his offspring to Molech, to defile my sanctuary, and to profane my holy name.
If the people of the land all hide their eyes from that person, when he gives of his offspring to Molech, and don’t put him to death;
then I will set my face against that man, and against his family, and will cut him off, and all who play the prostitute after him, to play the prostitute

He had thirty sons; and thirty daughters he sent outside his clan, and thirty daughters he brought in from outside his clan for his sons. He judged Israel seven years.
Ibzan died, and was buried at Bethlehem.
After him Elon the Zebulunite judged Israel; and he judged Israel ten years.
Elon the Zebulunite died, and was buried in Aijalon in the land of Zebulun.
After him Abdon the son of Hillel the Pirathonite judged Israel.
He had forty sons and thirty sons’ sons, who rode on seventy donkey colts. He judged Israel eight years.
Abdon the son of Hillel the Pirathonite died, and was buried in Pirathon in the land of Ephraim, in the hill country of the Amalekites.
The children of Israel again did that which was evil in the sight of Yahweh; and Yahweh delivered them into the hand of the Philistines forty years.
There was a certain man of Zorah, of the family of the Danites, whose name was Manoah; and his wife was barren, and childless.
Yahweh’s angel appeared to the woman, and said to her, “

chief Magdiel, chief Iram. These are the chiefs of Edom.
These are the sons of Israel: Reuben, Simeon, Levi, and Judah, Issachar, and Zebulun,
Dan, Joseph, and Benjamin, Naphtali, Gad, and Asher.
The sons of Judah: Er, and Onan, and Shelah; which three were born to him of Shua’s daughter the Canaanitess. Er, Judah’s firstborn, was wicked in the sight of Yahweh; and he killed him.
Tamar his daughter-in-law bore him Perez and Zerah. All the sons of Judah were five.
The sons of Perez: Hezron, and Hamul.
The sons of Zerah: Zimri, and Ethan, and Heman, and Calcol, and Dara; five of them in all.
The sons of Carmi: Achar, the troubler of Israel, who committed a trespass in the devoted thing.
The sons of Ethan: Azariah.
The sons also of Hezron, who were born to him: Jerahmeel, and Ram, and Chelubai.
Ram became the father of Amminadab, and Amminadab became the father of Nahshon, prince of the children of Judah;
and Nahshon became the father of Salma, and Salma became the father of Boaz,
and Boa

“Surely you have spoken in my hearing, I have heard the voice of your words, saying,
‘I am clean, without disobedience. I am innocent, neither is there iniquity in me.
Behold, he finds occasions against me. He counts me for his enemy.
He puts my feet in the stocks. He marks all my paths.’
“Behold, I will answer you. In this you are not just, for God is greater than man.
Why do you strive against him, because he doesn’t give account of any of his matters?
For God speaks once, yes twice, though man pays no attention.
In a dream, in a vision of the night, when deep sleep falls on men, in slumbering on the bed;
Then he opens the ears of men, and seals their instruction,
That he may withdraw man from his purpose, and hide pride from man.
He keeps back his soul from the pit, and his life from perishing by the sword.
He is chastened also with pain on his bed, with continual strife in his bones;
So that his life abhors bread, and his soul dainty food.
His flesh is so consumed away, that it can

Ah sinful nation, a people loaded with iniquity, offspring of evildoers, children who deal corruptly! They have forsaken Yahweh. They have despised the Holy One of Israel. They are estranged and backward.
Why should you be beaten more, that you revolt more and more? The whole head is sick, and the whole heart faint.
From the sole of the foot even to the head there is no soundness in it: wounds, welts, and open sores. They haven’t been closed, neither bandaged, neither soothed with oil.
Your country is desolate. Your cities are burned with fire. Strangers devour your land in your presence, and it is desolate, as overthrown by strangers.
The daughter of Zion is left like a shelter in a vineyard, like a hut in a field of melons, like a besieged city.
Unless Yahweh of Armies had left to us a very small remnant, we would have been as Sodom; we would have been like Gomorrah.
Hear Yahweh’s word, you rulers of Sodom! Listen to the law of our God, you people of Gomorrah!
“What are the multitude

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [82]:
len(list(bible.keys()))

31102

In [39]:
rec['chapter_id'].zfill(3)+rec['paragraph_number'].zfill(3)+rec['verse_id'].zfill(3)

'001001001'

In [34]:
a

[{'book_id': 'Matt',
  'book_name': 'Matyu',
  'book_order': '55',
  'chapter_id': '1',
  'chapter_title': 'Chapter 1',
  'paragraph_number': '1',
  'verse_id': '1',
  'verse_text': 'Jiysis Krays nop nusind yes nuk gok tikey amiy apiy gek nup tikyak rek aŋgniŋg gispiyn. Biyomb Ebraham tikek tik dand amiy, kiyŋ Depiyt nup tikeyak mey; Depiyt nund tikek tik dand amiy mey, Jiysis Krays nup tikyak. \n\t\t\t'},
 {'book_id': 'Matt',
  'book_name': 'Matyu',
  'book_order': '55',
  'chapter_id': '1',
  'chapter_title': 'Chapter 1',
  'paragraph_number': '1',
  'verse_id': '2',
  'verse_text': 'Kun giyak: Ebraham Aysek nup tikak. Aysek, Jekop nup tikak. Jekop nund pen, Jiwda nup tikiy, numam onep aŋgip ak tikiy gak. \n\t\t\t'},
 {'book_id': 'Matt',
  'book_name': 'Matyu',
  'book_order': '55',
  'chapter_id': '1',
  'chapter_title': 'Chapter 1',
  'paragraph_number': '1',
  'verse_id': '3',
  'verse_text': 'Jiwda nuk biyn Tema yip mindiy, Peres yip Sera yip kuyip tikak. Peres, Hesron nup tikak.